In [104]:
import pandas as pd
import numpy as np
import dataframe_image as dfi
import os

In [103]:
output_path = '../../data/output/'

In [5]:
# simulated data for widget A
df_a = pd.DataFrame(
    {
        'Month': pd.date_range(
            start = '01-01-2012',
            end = '31-12-2022',
            freq = 'MS'
        ),
        'Quotes': np.random.randint(
            low = 1_000_000,
            high = 2_500_000,
            size = 132
        ),
        'Numbers': np.random.randint(
            low = 300_000,
            high = 500_000,
            size = 132
        ),
        'Amounts': np.random.randint(
            low = 750_000,
            high = 1_250_000,
            size = 132
        )
    }
)

df_a['Product'] = 'A'

# simulated data for widget B
df_b = pd.DataFrame(
    {
        'Month':pd.date_range(
            start = '01-01-2012',
            end = '31-12-2022',
            freq = 'MS'
        ),
        'Quotes':np.random.randint(
            low = 100_000,
            high = 800_000,
            size = 132
        ),
        'Numbers':np.random.randint(
            low = 10_000,
            high = 95_000,
            size = 132
        ),
        'Amounts':np.random.randint(
            low = 450_000,
            high = 750_000,
            size = 132
        )
    }
)

df_b['Product'] = 'B'

# put it together & sort
df = pd.concat([df_a,df_b],axis = 0)
df.sort_values(by = 'Month',inplace = True)
df.reset_index(drop = True,inplace = True)
print(df.shape)
df.head()

(264, 5)


,Month,Quotes,Numbers,Amounts,Product
0,2012-01-01,1909815,310941,1032483,A
1,2012-01-01,403994,20569,716336,B
2,2012-02-01,1091805,318548,1000202,A
3,2012-02-01,734567,58324,519789,B
4,2012-03-01,1904792,354111,1086991,A


In [7]:
# average sale
df['Average sale'] = df['Amounts'] / df['Numbers']

# conversion
df['Product conversion'] = df['Numbers'] / df['Quotes']

df.head()

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,2012-01-01,1909815,310941,1032483,A,3.320511,0.162812
1,2012-01-01,403994,20569,716336,B,34.826000,0.050914
2,2012-02-01,1091805,318548,1000202,A,3.139878,0.291763
3,2012-02-01,734567,58324,519789,B,8.912095,0.079399
4,2012-03-01,1904792,354111,1086991,A,3.069634,0.185905


### Formatting dates

we can improve readability even further by using the name of each month rather than the month number and we can do this **without having to alter the underlying data**

In [17]:
# remove day of month from month column
df.loc[:5,:].style.format({'Month':'{:%Y-%m}'})

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,2012-01,1909815,310941,1032483,A,3.320511,0.162812
1,2012-01,403994,20569,716336,B,34.826000,0.050914
2,2012-02,1091805,318548,1000202,A,3.139878,0.291763
3,2012-02,734567,58324,519789,B,8.912095,0.079399
4,2012-03,1904792,354111,1086991,A,3.069634,0.185905
5,2012-03,782278,25885,478157,B,18.472359,0.033089


In [18]:
# use full name of month
df.loc[:5,:].style.format({'Month':'{:%B %Y}'})

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,January 2012,1909815,310941,1032483,A,3.320511,0.162812
1,January 2012,403994,20569,716336,B,34.826000,0.050914
2,February 2012,1091805,318548,1000202,A,3.139878,0.291763
3,February 2012,734567,58324,519789,B,8.912095,0.079399
4,March 2012,1904792,354111,1086991,A,3.069634,0.185905
5,March 2012,782278,25885,478157,B,18.472359,0.033089


In [19]:
# use abbreviated month name
df.loc[:5,:].style.format({'Month':'{:%b %Y}'})

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,Jan 2012,1909815,310941,1032483,A,3.320511,0.162812
1,Jan 2012,403994,20569,716336,B,34.826000,0.050914
2,Feb 2012,1091805,318548,1000202,A,3.139878,0.291763
3,Feb 2012,734567,58324,519789,B,8.912095,0.079399
4,Mar 2012,1904792,354111,1086991,A,3.069634,0.185905
5,Mar 2012,782278,25885,478157,B,18.472359,0.033089


In [20]:
# year and month number, separated by letter 'M'
df.loc[:5,:].style.format({'Month':'{:%Y M%m}'})

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,2012 M01,1909815,310941,1032483,A,3.320511,0.162812
1,2012 M01,403994,20569,716336,B,34.826000,0.050914
2,2012 M02,1091805,318548,1000202,A,3.139878,0.291763
3,2012 M02,734567,58324,519789,B,8.912095,0.079399
4,2012 M03,1904792,354111,1086991,A,3.069634,0.185905
5,2012 M03,782278,25885,478157,B,18.472359,0.033089


{:%Y M%#m} will remove leading zero

### Formatting numbers with a thousand separator

In [23]:
# thousands separator for absolute numbers
df.loc[:5,:].style.format(
    {
        'Month':'{:%b %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}'
    }
)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,Jan 2012,"1,909,815","310,941",1032483,A,3.320511,0.162812
1,Jan 2012,"403,994","20,569",716336,B,34.826000,0.050914
2,Feb 2012,"1,091,805","318,548",1000202,A,3.139878,0.291763
3,Feb 2012,"734,567","58,324",519789,B,8.912095,0.079399
4,Mar 2012,"1,904,792","354,111",1086991,A,3.069634,0.185905
5,Mar 2012,"782,278","25,885",478157,B,18.472359,0.033089


### Formatting currencies

In [25]:
# currency formatting
df.loc[:5,:].style.format(
    {
        'Month':'{:%b %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}',
        'Amounts':'${:,.0f}',
        'Average sale':'${:,.2f}'
    }
)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,Jan 2012,"1,909,815","310,941","$1,032,483",A,$3.32,0.162812
1,Jan 2012,"403,994","20,569","$716,336",B,$34.83,0.050914
2,Feb 2012,"1,091,805","318,548","$1,000,202",A,$3.14,0.291763
3,Feb 2012,"734,567","58,324","$519,789",B,$8.91,0.079399
4,Mar 2012,"1,904,792","354,111","$1,086,991",A,$3.07,0.185905
5,Mar 2012,"782,278","25,885","$478,157",B,$18.47,0.033089


In [26]:
# different currency representation
df.loc[:5,:].style.format(
    {
        'Month':'{:%b %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}',
        'Amounts':'${:,.0f}',
        'Average sale':'{:,.2f} ($)'
    }
)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,Jan 2012,"1,909,815","310,941","$1,032,483",A,3.32 ($),0.162812
1,Jan 2012,"403,994","20,569","$716,336",B,34.83 ($),0.050914
2,Feb 2012,"1,091,805","318,548","$1,000,202",A,3.14 ($),0.291763
3,Feb 2012,"734,567","58,324","$519,789",B,8.91 ($),0.079399
4,Mar 2012,"1,904,792","354,111","$1,086,991",A,3.07 ($),0.185905
5,Mar 2012,"782,278","25,885","$478,157",B,18.47 ($),0.033089


### Formatting percentages

In [27]:
# percentage formatting
df.loc[:5,:].style.format(
    {
        'Month':'{:%b %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}',
        'Amounts':'${:,.0f}',
        'Average sale':'${:,.2f}',
        'Product conversion':'{:.2%}'
    }
)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,Jan 2012,"1,909,815","310,941","$1,032,483",A,$3.32,16.28%
1,Jan 2012,"403,994","20,569","$716,336",B,$34.83,5.09%
2,Feb 2012,"1,091,805","318,548","$1,000,202",A,$3.14,29.18%
3,Feb 2012,"734,567","58,324","$519,789",B,$8.91,7.94%
4,Mar 2012,"1,904,792","354,111","$1,086,991",A,$3.07,18.59%
5,Mar 2012,"782,278","25,885","$478,157",B,$18.47,3.31%


### Hide the index

In [29]:
# suppress the index
df.loc[:5,:].style.format(
    {
        'Month':'{:%b %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}',
        'Amounts':'${:,.0f}',
        'Average sale':'${:,.2f}',
        'Product conversion':'{:.2%}'
    }
).hide()
# ).hide_index()

Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
Jan 2012,"1,909,815","310,941","$1,032,483",A,$3.32,16.28%
Jan 2012,"403,994","20,569","$716,336",B,$34.83,5.09%
Feb 2012,"1,091,805","318,548","$1,000,202",A,$3.14,29.18%
Feb 2012,"734,567","58,324","$519,789",B,$8.91,7.94%
Mar 2012,"1,904,792","354,111","$1,086,991",A,$3.07,18.59%
Mar 2012,"782,278","25,885","$478,157",B,$18.47,3.31%


### Conditional formatting

In [41]:
# function to conditionally highlight rows based on product
def highlight_product(s, product, color):
    r = pd.Series(data = False,index = s.index)
    r['Product'] = s.loc['Product'] == product
    
    return [f'background-color: {color}' if r.any() else '' for v in r]

In [45]:
color = 'gray'

# apply the formatting
df.loc[:5,:].style\
    .apply(highlight_product, product = 'A', color=color, axis = 1)\
    .format(
        {
            'Month':'{:%b %Y}',
            'Quotes':'{:,.0f}',
            'Numbers':'{:,.0f}',
            'Amounts':'${:,.0f}',
            'Average sale':'${:,.2f}',
            'Product conversion':'{:.2%}'
        }
).hide()

Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
Jan 2012,"1,909,815","310,941","$1,032,483",A,$3.32,16.28%
Jan 2012,"403,994","20,569","$716,336",B,$34.83,5.09%
Feb 2012,"1,091,805","318,548","$1,000,202",A,$3.14,29.18%
Feb 2012,"734,567","58,324","$519,789",B,$8.91,7.94%
Mar 2012,"1,904,792","354,111","$1,086,991",A,$3.07,18.59%
Mar 2012,"782,278","25,885","$478,157",B,$18.47,3.31%


we can provide colour hex codes to pandas

In [50]:
# function to highlight rows based on average sale
def highlight_average_sale(s, color, sale_threshold = 5):
    r = pd.Series(data = False,index = s.index)
    r['Product'] = s.loc['Average sale'] > sale_threshold
    
    return [f'background-color: {color}' if r.any() else '' for v in r]

In [52]:
color = 'gray'

# apply the formatting
df.iloc[:5,:].style\
    .apply(highlight_average_sale, color=color, sale_threshold = 20, axis = 1)\
    .format(
        {
            'Month':'{:%b %Y}',
            'Quotes':'{:,.0f}',
            'Numbers':'{:,.0f}',
            'Amounts':'${:,.0f}',
            'Average sale':'${:,.2f}',
            'Product conversion':'{:.2%}'
        }
).hide()

Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
Jan 2012,"1,909,815","310,941","$1,032,483",A,$3.32,16.28%
Jan 2012,"403,994","20,569","$716,336",B,$34.83,5.09%
Feb 2012,"1,091,805","318,548","$1,000,202",A,$3.14,29.18%
Feb 2012,"734,567","58,324","$519,789",B,$8.91,7.94%
Mar 2012,"1,904,792","354,111","$1,086,991",A,$3.07,18.59%


In [55]:
# functions to change font colour based on a threshold
def color_threshold_lessthan(value, threshold, color = 'red'):
    if value < threshold:
        return f'color: {color}'
    else:
        return ''
    
def color_threshold_morethan(value,threshold, color = 'green'):
    if value > threshold:
        return f'color: {color}'
    else:
        return ''

# functions to change font weight based on a threshold    
def weight_threshold_lessthan(value, threshold):
    if value < threshold:
        return f'font-weight: bold'
    else:
        return ''

def weight_threshold_morethan(value, threshold):
    if value > threshold:
        return f'font-weight: bold'
    else:
        return ''

In [62]:
color = 'gray'

# apply the formatting
df.iloc[:10,:].style\
    .apply(highlight_product,product = 'A',color = color, axis = 1)\
    .map(color_threshold_lessthan,threshold = 0.05,subset = ['Product conversion'])\
    .map(weight_threshold_lessthan,threshold = 0.05,subset = ['Product conversion'])\
    .map(color_threshold_morethan,threshold = 0.2,subset = ['Product conversion'])\
    .map(weight_threshold_morethan,threshold = 0.2,subset = ['Product conversion'])\
    .format(
        {
            'Month':'{:%b %Y}',
            'Quotes':'{:,.0f}',
            'Numbers':'{:,.0f}',
            'Amounts':'${:,.0f}',
            'Average sale':'${:,.2f}',
            'Product conversion':'{:.2%}'
        }
).hide()

Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
Jan 2012,"1,909,815","310,941","$1,032,483",A,$3.32,16.28%
Jan 2012,"403,994","20,569","$716,336",B,$34.83,5.09%
Feb 2012,"1,091,805","318,548","$1,000,202",A,$3.14,29.18%
Feb 2012,"734,567","58,324","$519,789",B,$8.91,7.94%
Mar 2012,"1,904,792","354,111","$1,086,991",A,$3.07,18.59%
Mar 2012,"782,278","25,885","$478,157",B,$18.47,3.31%
Apr 2012,"2,298,145","352,308","$1,072,301",A,$3.04,15.33%
Apr 2012,"621,296","64,766","$504,539",B,$7.79,10.42%
May 2012,"2,040,984","489,371","$1,006,599",A,$2.06,23.98%
May 2012,"335,095","71,549","$723,404",B,$10.11,21.35%


### Table level changes: text alignment and captions

In [79]:
color = 'gray'

# apply the formatting
df.iloc[:10,:].style\
    .set_properties(**{'text-align':'center'})\
    .apply(highlight_product,product = 'A',color = color, axis = 1)\
    .map(color_threshold_lessthan,threshold = 0.15,subset = ['Product conversion'])\
    .map(weight_threshold_lessthan,threshold = 0.15,subset = ['Product conversion'])\
    .map(color_threshold_morethan,threshold = 0.2,subset = ['Product conversion'])\
    .map(weight_threshold_morethan,threshold = 0.2,subset = ['Product conversion'])\
    .format(
        {
            'Month':'{:%b %Y}',
            'Quotes':'{:,.0f}',
            'Numbers':'{:,.0f}',
            'Amounts':'${:,.0f}',
            'Average sale':'${:,.2f}',
            'Product conversion':'{:.2%}'
        }
    ).set_caption('Sales data <br> Produced by Team X')\
.hide()

Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
Jan 2012,"1,909,815","310,941","$1,032,483",A,$3.32,16.28%
Jan 2012,"403,994","20,569","$716,336",B,$34.83,5.09%
Feb 2012,"1,091,805","318,548","$1,000,202",A,$3.14,29.18%
Feb 2012,"734,567","58,324","$519,789",B,$8.91,7.94%
Mar 2012,"1,904,792","354,111","$1,086,991",A,$3.07,18.59%
Mar 2012,"782,278","25,885","$478,157",B,$18.47,3.31%
Apr 2012,"2,298,145","352,308","$1,072,301",A,$3.04,15.33%
Apr 2012,"621,296","64,766","$504,539",B,$7.79,10.42%
May 2012,"2,040,984","489,371","$1,006,599",A,$2.06,23.98%
May 2012,"335,095","71,549","$723,404",B,$10.11,21.35%


### Putting it all together

In [89]:
# create a total "row" - i.e. column total

total = df.sum(numeric_only=True)
# total['Month'] = pd.NaT
total['Product'] = ''
total['Average sale'] = total['Amounts'] / total['Numbers']
total['Product conversion'] = total['Numbers'] / total['Quotes']
total = total.to_frame().transpose()

In [82]:
# function to highlight the total row
def highlight_total(s):
    r = pd.Series(data = False,index = s.index)
    r['Month'] = pd.isnull(s.loc['Month'])
    
    return ['font-weight: bold' if r.any() else '' for v in r]

In [100]:
color = 'gray'

#stack and reset index
d = pd.concat([df,total],axis = 0)
d.reset_index(drop = True,inplace = True)

# apply formatting
d.iloc[-10:,:].style\
    .set_properties(**{'text-align':'center'})\
    .apply(highlight_product, product = 'A', color = color, axis = 1)\
    .apply(highlight_total,axis = 1)\
    .format(
        {
            'Month':'{:%b %Y}',
            'Quotes':'{:,.0f}',
            'Numbers':'{:,.0f}',
            'Amounts':'${:,.0f}',
            'Average sale':'${:,.2f}',
            'Product conversion':'{:.2%}'
        },
        na_rep = 'Total'
    ).set_caption('Sales data <br> Produced by Team X')\
    .hide()

Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
Aug 2022,"1,361,092","364,565","$1,045,055",A,$2.87,26.78%
Sep 2022,"364,249","29,838","$516,893",B,$17.32,8.19%
Sep 2022,"1,163,255","374,764","$992,195",A,$2.65,32.22%
Oct 2022,"569,453","78,265","$699,224",B,$8.93,13.74%
Oct 2022,"1,532,336","339,258","$1,228,426",A,$3.62,22.14%
Nov 2022,"119,195","38,798","$696,098",B,$17.94,32.55%
Nov 2022,"1,077,135","316,514","$853,908",A,$2.70,29.38%
Dec 2022,"1,487,631","369,639","$887,993",A,$2.40,24.85%
Dec 2022,"379,001","55,156","$608,116",B,$11.03,14.55%
Total,"290,548,836","59,381,802","$209,661,914",,$3.53,20.44%


### Export the magic

In [110]:
color = 'gray'

# style the table
d_styled = d.style\
    .set_properties(**{'text-align':'center'})\
    .apply(highlight_product, product = 'A', color = color, axis = 1)\
    .apply(highlight_total,axis = 1)\
    .format(
        {
            'Month':'{:%b %Y}',
            'Quotes':'{:,.0f}',
            'Numbers':'{:,.0f}',
            'Amounts':'${:,.0f}',
            'Average sale':'${:,.2f}',
            'Product conversion':'{:.2%}'
        },
        na_rep = 'Total'
    ).set_caption('Sales data <br> Produced by Team X')\
    .hide()

# export the table to PNG
dfi.export(
    d_styled,
    os.path.join(
        output_path,
        'styled_dataframe.png'
    ),
    max_rows=-1
)